---
title: 链家网房源
date: 2019-07-05
categories: [爬虫开发, 项目]
---

In [7]:
import requests
import time
from bs4 import BeautifulSoup

## request

### 设置列表页URL的固定部分

In [8]:
url = 'http://bj.lianjia.com/ershoufang/'

#设置页面页的可变部分
page = ('pg')

### 设置请求头部信息

In [9]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
'Accept':'text/html;q=0.9,*/*;q=0.8',
'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
'Accept-Encoding':'gzip',
'Connection':'close',
'Referer':'http://www.baidu.com/link?url=_andhfsjjjKRgEWkj7i9cFmYYGsisrnm2A-TN3XZDQXxvGsM9k9ZZSnikW2Yds4s&amp;amp;wd=&amp;amp;eqid=c3435a7d00146bd600000003582bfd1f'
}

### 循环抓取列表页信息

In [10]:
for i in range(1,2):
     if i == 1:
          i=str(i)
          a=(url+page+i+'/')
          r=requests.get(url=a,headers=headers)
          html=r.content
     else:
          i=str(i)
          a=(url+page+i+'/')
          r=requests.get(url=a,headers=headers)
          html2=r.content
          html = html + html2
     #每次间隔1秒
     time.sleep(1)

### 解析抓取的页面内容

In [11]:
lj=BeautifulSoup(html,'html.parser')

### 提取房源总价

In [12]:
price=lj.find_all('div','priceInfo')
tp=[]
for a in price:
    totalPrice=a.span.string
    tp.append(totalPrice)

### 提取房源信息

In [13]:
houseInfo=lj.find_all('div',attrs={'class':'houseInfo'})
 
hi=[]
for b in houseInfo:
    house=b.get_text()
    hi.append(house)

### 提取房源关注度

In [14]:
followInfo=lj.find_all('div',attrs={'class':'followInfo'})
 
fi=[]
for c in followInfo:
    follow=c.get_text()
    fi.append(follow)

### DataFrame

In [30]:
import pandas as pd
house=pd.DataFrame({'totalprice':tp,'houseinfo':hi,'followinfo':fi})
house.head()

,totalprice,houseinfo,followinfo
0,268,农光东里 | 1室1厅 | 37.55平米 | 南 | 精装,150人关注 / 6天以前发布
1,381,平乐园小区 | 2室1厅 | 60.52平米 | 南 北 | 精装,135人关注 / 18天以前发布
2,570,科源小区 | 2室1厅 | 65.1平米 | 东 西 | 简装,70人关注 / 5个月以前发布
3,365,劲松南路 | 2室1厅 | 57.49平米 | 南 北 | 简装,97人关注 / 10天以前发布
4,688,大井胡同 | 1室1厅 | 38平米 | 南 | 精装,18人关注 / 12天以前发布


### 对房源信息进行分列

In [31]:
houseinfo_split = pd.DataFrame((x.split('|') for x in house.houseinfo),index=house.index,columns=['小区','户型','面积','朝向','装修'])
houseinfo_split.head()

,小区,户型,面积,朝向,装修
0,农光东里,1室1厅,37.55平米,南,精装
1,平乐园小区,2室1厅,60.52平米,南 北,精装
2,科源小区,2室1厅,65.1平米,东 西,简装
3,劲松南路,2室1厅,57.49平米,南 北,简装
4,大井胡同,1室1厅,38平米,南,精装


### 将分列结果拼接回原数据表

In [32]:
house=pd.merge(house,houseinfo_split,right_index=True, left_index=True)
house.head()

,totalprice,houseinfo,followinfo,小区,户型,面积,朝向,装修
0,268,农光东里 | 1室1厅 | 37.55平米 | 南 | 精装,150人关注 / 6天以前发布,农光东里,1室1厅,37.55平米,南,精装
1,381,平乐园小区 | 2室1厅 | 60.52平米 | 南 北 | 精装,135人关注 / 18天以前发布,平乐园小区,2室1厅,60.52平米,南 北,精装
2,570,科源小区 | 2室1厅 | 65.1平米 | 东 西 | 简装,70人关注 / 5个月以前发布,科源小区,2室1厅,65.1平米,东 西,简装
3,365,劲松南路 | 2室1厅 | 57.49平米 | 南 北 | 简装,97人关注 / 10天以前发布,劲松南路,2室1厅,57.49平米,南 北,简装
4,688,大井胡同 | 1室1厅 | 38平米 | 南 | 精装,18人关注 / 12天以前发布,大井胡同,1室1厅,38平米,南,精装


### 对房源关注度进行分列

In [33]:
followinfo_split = pd.DataFrame((x.split('/') for x in house.followinfo),index=house.index,columns=['guanzhu','fabu'])
house=pd.merge(house,followinfo_split,right_index=True, left_index=True)
house.head()

,totalprice,houseinfo,followinfo,小区,户型,面积,朝向,装修,guanzhu,fabu
0,268,农光东里 | 1室1厅 | 37.55平米 | 南 | 精装,150人关注 / 6天以前发布,农光东里,1室1厅,37.55平米,南,精装,150人关注,6天以前发布
1,381,平乐园小区 | 2室1厅 | 60.52平米 | 南 北 | 精装,135人关注 / 18天以前发布,平乐园小区,2室1厅,60.52平米,南 北,精装,135人关注,18天以前发布
2,570,科源小区 | 2室1厅 | 65.1平米 | 东 西 | 简装,70人关注 / 5个月以前发布,科源小区,2室1厅,65.1平米,东 西,简装,70人关注,5个月以前发布
3,365,劲松南路 | 2室1厅 | 57.49平米 | 南 北 | 简装,97人关注 / 10天以前发布,劲松南路,2室1厅,57.49平米,南 北,简装,97人关注,10天以前发布
4,688,大井胡同 | 1室1厅 | 38平米 | 南 | 精装,18人关注 / 12天以前发布,大井胡同,1室1厅,38平米,南,精装,18人关注,12天以前发布


> 参考：

1. [廖雪峰Python数据分析](https://www.julyedu.com/course/getDetail/66/)